# ERC-4626: vault ecosystem comparison across chains

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- Read general instructions [how to run the tutorials](./)
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [68]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [69]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 7017 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [70]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")

df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())



DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Protocol, Denomination, NAV, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol  \
Chain    Address                                                
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  yvUSDC-A   
         0xbb287e6017d3deb0e2e65061e8684eab21060123  yvUSDT-A   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72    pufETH   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1     mwETH   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688     fWETH   

                                                                      Name  \
Chain    Address                                                             
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0          USDC yVault-A   
         0xbb287e6017d3deb0e2e65061e8684eab21060123          USDT yVault-A   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72                 pufETH   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1  Moonwell Flagship ETH   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688              FARM_WETH   

                                                            Protocol  \
Chain    Address                                                       
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0   <generic 4626>   
         0xbb287e6017d3deb0e2e65061e8684eab21060123   <generic 4626>   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72   <generic 4626>   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1           Morpho   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688  Harvest Finance   

                                                    Denomination        NAV  \
Chain    Address                                                              
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0         USDC 571,093.38   
         0xbb287e6017d3deb0e2e65061e8684eab21060123         USDT 462,260.13   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72         WETH  70,733.48   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1         WETH  13,598.65   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688         WETH   1,097.20   

                                                      Mgmt fee   Perf fee  \
Chain    Address                                                            
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  <unknown>  <unknown>   
         0xbb287e6017d3deb0e2e65061e8684eab21060123  <unknown>  <unknown>   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72  <unknown>  <unknown>   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1       0.00       0.00   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688  <unknown>  <unknown>   

                                                                       Shares  \
Chain    Address                                                                
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0             484856.813895   
         0xbb287e6017d3deb0e2e65061e8684eab21060123             390070.385481   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72  67756.256788011447280782   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1  13471.413060524250815073   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688    922.037629422087573134   

                                                             First seen  \
Chain    Address                                                          
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0 2023-10-26 16:45:48   
         0xbb287e6017d3deb0e2e65061e8684eab21060123 2023-12-07 17:02:20   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72 2024-01-31 19:44:35   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1 2024-06-12 12:57:15   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688 2023-09-07 00:27:03   

                                                                                       _detection_data  \
Chain    Address                                                                                         
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  ERC4262VaultDetection(chain=137, address='0xa0...   
         0xbb287e6017d3deb0e2e65061e8684e

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [71]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby(level='Chain').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby(level='Chain')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby(level='Chain').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby(level='Chain').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby(level='Chain').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby(level='Chain').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby(level='Chain').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby(level='Chain').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,1916,"115,751,031.41",265.00,659,55.00,18.00,26.00,219.00
Avalanche,259,"2,442,084.06",8.00,57,2.00,1.00,1.00,48.00
Base,1154,"296,067,389.20",565.00,420,48.00,24.00,12.00,93.00
Berachain,231,"345,279,363.27",1.00,17,11.00,0.00,0.00,2.00
Binance,316,"220,814,702.13",9.00,84,9.00,0.00,0.00,25.00
Ethereum,2115,"9,336,687,954.19",274.00,701,177.00,16.00,14.00,177.00
Hyperliquid,12,"110,083.46",0.00,6,1.00,1.00,0.00,1.00
Mantle,37,360.40,0.00,11,0.00,0.00,0.00,5.00
Mode,56,"449,480.93",0.00,26,2.00,0.00,0.00,0.00


# Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [72]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby('Protocol').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby('Protocol')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby('Protocol').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby('Protocol').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby('Protocol').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby('Protocol').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby('Protocol').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby('Protocol').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<generic 4626>,5416,"8,464,984,930.82",65.00,"1,865.00",223.00,0.00,12.00,0.00
Morpho,183,"1,366,664,229.93",183.00,93.00,56.00,0.00,0.00,183.00
Fluid,57,"492,349,554.91",57.00,25.00,7.00,0.00,0.00,0.00
Euler,37,"22,505,948.99",37.00,8.00,3.00,0.00,0.00,0.00
Kiln Metavault,68,"19,168,170.69",68.00,60.00,8.00,0.00,0.00,0.00
Gains Network,35,"10,147,498.09",35.00,14.00,10.00,3.00,0.00,0.00
IPOR,26,"6,406,042.09",26.00,21.00,5.00,0.00,0.00,26.00
Arcadia Finance,9,"4,625,452.75",9.00,3.00,2.00,0.00,0.00,0.00
Term Finance,49,"4,117,585.34",49.00,24.00,1.00,0.00,0.00,0.00


## Vault deployment history

- Show how much history we have for each chain


In [ ]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level='Chain')['First seen'].agg(['min', 'max']).reset_index()

# Rename columns for clarity
seen_df.columns = ['Chain', 'First vault deployed', 'Last vault deployed']

seen_df = seen_df.set_index("Chain")

display(seen_df)

ValueError: level name Chain is not the name of the index

## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [ ]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100] 
largest_df = largest_df[largest_df["Stablecoin denominated"] == True] 
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))


,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Savings USDS,"3,037,173,933.35",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,27690,<unknown>,<unknown>
Staked USDe,"2,180,494,599.88",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,69128,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"913,834,025.77",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,48138,<unknown>,<unknown>
Savings Dai,"516,290,716.10",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,72384,<unknown>,<unknown>
Bridged USDC (Stargate)Vault,"331,700,876.01",Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,129613,<unknown>,<unknown>
Usual Boosted USDC,"217,812,829.88",Ethereum,0xd63070114470f685b75b74d60eec7c1113d33a3d,USDC,17363,0.00,0.00
Fluid USD Coin,"216,518,662.58",Ethereum,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,USDC,17475,<unknown>,<unknown>
Staked USDX,"199,779,134.12",Binance,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5423,<unknown>,<unknown>
Fluid Tether USD,"196,596,047.45",Ethereum,0x5c20b550819128074fd538edf79791733ccedd18,USDT,10343,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [ ]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby('Chain')['NAV'].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

NAV  \
Chain     Name                                            
Arbitrum  Fluid USD Coin                  28,737,895.95   
Avalanche HiYield Treasury Bill Vault      2,274,986.21   
Base      Spark USDC Vault                53,841,156.87   
          Spark USDC Vault                 6,315,795.39   
Berachain Bridged USDC (Stargate)Vault   331,700,876.01   
Binance   Staked USDX                    199,779,134.12   
Ethereum  Savings USDS                 3,037,173,933.35   
Mode      USDC Ironclad Vault                337,647.27   
Polygon   Compound USDC                   24,099,469.86   

                                                                           Address  \
Chain     Name                                                                       
Arbitrum  Fluid USD Coin                0x1a996cb54bb95462040408c06122d45d6cdb6096   
Avalanche HiYield Treasury Bill Vault   0x8475509d391e6ee5a8b7133221ce17019d307b3e   
Base      Spark USDC Vault              0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a   
          Spark USDC Vault              0x3128a0f7f0ea68e7b7c9b00afa7e41045828e858   
Berachain Bridged USDC (Stargate)Vault  0x90bc07408f5b5eac4de38af76ea6069e1fcee363   
Binance   Staked USDX                   0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92   
Ethereum  Savings USDS                  0xa3931d71877c0e7a3148cb7eb4463524fec27fbd   
Mode      USDC Ironclad Vault           0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c   
Polygon   Compound USDC                 0x781fb7f6d845e3be129289833b04d43aa8558c42   

                                       Denomination  Total events   Mgmt fee  \
Chain     Name                                                                 
Arbitrum  Fluid USD Coin                       USDC         33551  <unknown>   
Avalanche HiYield Treasury Bill Vault          USDC           177  <unknown>   
Base      Spark USDC Vault                     USDC          5731       0.00   
          Spark USDC Vault                     USDC         31199  <unknown>   
Berachain Bridged USDC (Stargate)Vault       USDC.e        129613  <unknown>   
Binance   Staked USDX                          USDX          5423  <unknown>   
Ethereum  Savings USDS                         USDS         27690  <unknown>   
Mode      USDC Ironclad Vault                  USDC           183  <unknown>   
Polygon   Compound USDC                        USDC          4090       0.00   

                                         Perf fee  
Chain     Name                                     
Arbitrum  Fluid USD Coin                <unknown>  
Avalanche HiYield Treasury Bill Vault   <unknown>  
Base      Spark USDC Vault                   0.00  
          Spark USDC Vault              <unknown>  
Berachain Bridged USDC (Stargate)Vault  <unknown>  
Binance   Staked USDX                   <unknown>  
Ethereum  Savings USDS                  <unknown>  
Mode      USDC Ironclad Vault           <unknown>  
Polygon   Compound USDC                      0.00

## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [ ]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
USDC yVault-A,490167,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"571,093.38",530 days 15:45:30.767743,465206,24961
USDT yVault-A,458203,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"462,260.13",488 days 15:28:58.767743,407690,50513
pufETH,167818,Ethereum,0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"70,733.48",433 days 12:46:43.767743,151538,16280
Moonwell Flagship ETH,140531,Base,0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"13,598.65",300 days 19:34:03.767743,85380,55151
FARM_WETH,134053,Base,0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688,WETH,"1,097.20",580 days 08:04:15.767743,76070,57983
Bridged USDC (Stargate)Vault,129613,Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"331,700,876.01",73 days 08:25:15.767743,80477,49136
Staked USDA,121755,Arbitrum,0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"913,452.65",453 days 23:58:53.767743,119741,2014
Moonwell Flagship USDC,92152,Base,0xc1256ae5ff1cf2719d4937adb3bbccab2e00a2ca,USDC,"26,970,959.20",300 days 19:34:03.767743,53216,38936
Beraborrow iBGT,89520,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"123,531.01",32 days 20:25:59.767743,48361,41159


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [ ]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby('Chain')['Total events'].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"121,755.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"913,452.65",453 days 23:58:53.767743,119741,2014
Avalanche,GoGoPool Liquid Staking Token,"66,463.00",0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,WAVAX,"832,304.08",734 days 11:20:33.767743,41197,25266
Base,Moonwell Flagship ETH,"140,531.00",0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"13,598.65",300 days 19:34:03.767743,85380,55151
Berachain,Bridged USDC (Stargate)Vault,"129,613.00",0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"331,700,876.01",73 days 08:25:15.767743,80477,49136
Binance,kUSDT,"42,614.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"2,712,184.69",667 days 01:22:35.767743,25239,17375
Ethereum,pufETH,"167,818.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"70,733.48",433 days 12:46:43.767743,151538,16280
Hyperliquid,wHYPE,534.00,0x2831775cb5e64b1d892853893858a261e898fbeb,WHYPE,"167,349.65",27 days 10:20:18.767743,411,123
Mantle,Karak - mETH,"13,024.00",0x8529019503c5bd707d8eb98c5c87bf5237f89135,mETH,490.09,350 days 14:19:12.767743,8038,4986
Mode,Renzo aggregator,"21,417.00",0xd60dd6981ec336fda40820f8ca5e99cd17dd25a0,WETH,209.76,398 days 08:32:35.767743,12187,9230


## Oldest vaults

- Show oldest vaults

In [ ]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby('Chain')['Age'].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1047 days 17:16:12.767743,WETH,18.00,"2,335.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1091 days 15:33:56.767743,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,586 days 16:37:25.767743,WETH,0.97,"1,117.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,73 days 08:25:15.767743,USDC.e,"331,700,876.01","129,613.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1016 days 11:38:39.767743,NRCH,"1,188,908.73","1,195.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1079 days 21:05:41.767743,THOR,"79,360,216.48","19,570.00"
Mantle,0x8529019503c5bd707d8eb98c5c87bf5237f89135,Karak - mETH,350 days 14:19:12.767743,mETH,490.09,"13,024.00"
Mode,0xd60dd6981ec336fda40820f8ca5e99cd17dd25a0,Renzo aggregator,398 days 08:32:35.767743,WETH,209.76,"21,417.00"
Polygon,0x73958d46b7aa2bc94926d8a215fa560a5cdca3ea,Wrapped Aave Polygon GHST,1071 days 04:44:29.767743,aPolGHST,"1,144,866.24","14,510.00"
